In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import *
from math import *
from numpy.linalg import norm
%matplotlib inline
import matplotlib.pyplot as plt
import datetime

C:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarnin

In [2]:
train = pd.read_csv('train.csv', delimiter=',')

#1
#weekDay = train['datetime'].dt.dayofweek

#2
weekDay = np.arange(train.shape[0])
for idx, row in train.iterrows():
    #weekDay[idx] = row['datetime'].dt.dayofweek
    weekDay[idx] = pd.to_datetime(row['datetime'], coerce=True).weekday() #to_datetime, т.к row возвращает str


train = pd.read_csv('train.csv', delimiter=',', index_col='datetime', parse_dates=True)

train['year'] = train.index.year
train['month'] = train.index.month
train['hour'] = train.index.hour
#train['day'] = train.index.day
train['day'] = weekDay

season = train['month']
season = season.apply(lambda s: 4 if int(s) == 1 or int(s) == 2 else (int(s) - 3) // 3 + 1)   
train = train.drop('season', axis = 1)
train.insert(5, 'season', season)

hour = train['hour']
mainHour = np.arange(hour.shape[0])
mainHour = [1 if (hour[i] == 7) or (hour[i] == 8) or (hour[i] == 17) or (hour[i] == 18) else 0 for i in range(hour.shape[0])]
train.insert(6, 'mainHour', mainHour)

#print (datetime.datetime.today().weekday())
train.head(50)

C:\Program Files (x86)\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: FutureWarning: the coerce=True keyword is deprecated, use errors='coerce' instead


,holiday,workingday,weather,temp,atemp,season,mainHour,humidity,windspeed,casual,registered,count,year,month,hour,day
datetime,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,0,0,1,9.84,14.395,4,0,81,0.0000,3,13,16,2011,1,0,5
2011-01-01 01:00:00,0,0,1,9.02,13.635,4,0,80,0.0000,8,32,40,2011,1,1,5
2011-01-01 02:00:00,0,0,1,9.02,13.635,4,0,80,0.0000,5,27,32,2011,1,2,5
2011-01-01 03:00:00,0,0,1,9.84,14.395,4,0,75,0.0000,3,10,13,2011,1,3,5
2011-01-01 04:00:00,0,0,1,9.84,14.395,4,0,75,0.0000,0,1,1,2011,1,4,5
2011-01-01 05:00:00,0,0,2,9.84,12.880,4,0,75,6.0032,0,1,1,2011,1,5,5
2011-01-01 06:00:00,0,0,1,9.02,13.635,4,0,80,0.0000,2,0,2,2011,1,6,5
2011-01-01 07:00:00,0,0,1,8.20,12.880,4,1,86,0.0000,1,2,3,2011,1,7,5
2011-01-01 08:00:00,0,0,1,9.84,14.395,4,1,75,0.0000,1,7,8,2011,1,8,5
